In [1]:
import numpy as np
import random as rd

In [2]:
N=100
G=4.302*10**-3 #pc (km/s)^2 Msun^-1
size = 50
levels = int(np.floor(np.log(N)/np.log(4)))
eps=2
p = int(np.log2(eps))
numcells = 4**levels
rd.seed(10)

In [3]:
mass = np.ones((N)).reshape(-1,1)
x = [rd.uniform(-size,size) for i in range(N)]
y = [rd.uniform(-size,size) for i in range(N)]
points = np.array(list(zip(x,y)))

In [47]:
import pdb

def cellPos(coords, cells): #Return the cell each particle belongs to, as well as the midpoint of the cell.
    pdb.set_trace()
    cellmid = cells[1::2]
    cellbound = cells[::2]
    for i in range(len(cellbound)):
        if coords[0] > cellbound[i]:
            midx = cellmid[i-1]
            numx = i
            break
    for i in range(len(cellbound)):
        if coords[1] > cellbound[i]:
            midy = cellmid[i-1]
            numy = i
            break
    return [numx,numy,complex(midx,midy)]

In [48]:
def coeff(z, q, k): #return expansion coefficients
    #a = np.zeros(p, complex)
    a = [-q[m]*(z[m]**k)/k for m in range(len(q))]
    #for m in range(len(q)):
    #a[k-1] = -q[m]*(z[m]**k)/k
    return np.sum(a)

def potentialStep1(z,z0,q): #calculate potential centered around cell center, z0
    t1 = np.sum(q)*np.log(z0)
    t2 = np.sum([coeff(z-z0,q,k)/(z0**k) for k in range(1,p+1)])
    return t1+t2

def upPass1(points, mass):
    cells = np.linspace(size,-size, 1+2**(levels+1)) #Mesh coordinates for finest mesh [level+1] (to make finding midpoints easier)
    finest_pos = np.array([cellPos(coords, cells) for coords in points]) #Midpoints & cell numbers for points at finest mesh level
    data = 1j*points[:,1]; data+=points[:,0]; data = data.reshape(N,1)
    data = np.append(data,finest_pos, axis=1) #Total data in complex form (coordinates, cell numbers, midpoints)
    for i in range(1,2**levels):
        for j in range(1,2**levels):
            pointfilter = np.where( (data[:,1] == i) & (data[:,2] == j) )
            try:
                z = data[:,0][pointfilter]
                z0 = data[:,3][pointfilter][0]
                q = mass[pointfilter]
                print(potentialStep1(z,z0,q))
            except:
                continue
                
def data(points):
    for l in range(levels+1):
        pdb.set_trace()
        cells = np.linspace(size,-size, 1+2**(l+1))
        pos = np.array([cellPos(coords, cells) for coords in points])
        data = np.append(data, pos, axis=1)

def upPass2(points, mass):
    l = levels - 1
    while(l >= 0):
        cells = np.linspace(size,-size,1+2**(levels+2))
        #mesh_pos = 

In [49]:
data(points)

> <ipython-input-48-940d51ca6bc9>(34)data()
-> data = np.append(data, pos, axis=1)
(Pdb) pos
array([[ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       [ 1.+0.j,  1.+0.j,  0.+0.j],
       

BdbQuit: 